<div style="width:100%;text-align: center;">
    <img src="https://user.oc-static.com/upload/2019/02/24/15510245026714_Seattle_logo_landscape_blue-black.png" />
</div>

# Introduction

Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués par vos agents en 2015 et en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments pour lesquels elles n’ont pas encore été mesurées.

Votre prédiction se basera sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction..)

Vous cherchez également à évaluer l’intérêt de l’"ENERGY STAR Score" pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe.


# Sommaire

1. [Chargement des données](#data-loading)  
2. [Comparaison des modèles](#model-compare)  
3. [Comparaison des modèles sans le EnergyStarScore](#model-compare-without-ESS)  
4. [Conclusion](#conclude)

In [ ]:
# import des bibliothèques utilisées
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, SGDRegressor, ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import *
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor


# <a id="data-loading">Chargement des données<a/> 

In [ ]:
# chargement des données 
data = pd.read_csv("/kaggle/input/refined-data/data_hard_refined.csv")
objectColumns = list(data.dtypes[data.dtypes == object].index)
numericColumns = list(data.dtypes[data.dtypes != object].index)
data = data.drop(['PropertyName', 'Address','OSEBuildingID', 'TaxParcelIdentificationNumber'], axis=1)
objectColumns = list(data.dtypes[data.dtypes == object].index)
numericColumns = list(data.dtypes[data.dtypes != object].index)
y_columns = ['TotalGHGEmissions', 'SiteEnergyUse(kBtu)']
X = data.drop(y_columns, axis=1)
y = data[y_columns]
for i in y_columns:
    numericColumns.remove(i)

In [ ]:
# standardiser les données
preprocessor = make_column_transformer((RobustScaler(),numericColumns),(OneHotEncoder(handle_unknown = 'ignore'),objectColumns))
results = []

In [ ]:
X_train, X_test, y_train_all, y_test_all = train_test_split(X, y, test_size=0.2, random_state=2)

In [ ]:
algos = {
    'LinearRegression' : {
        'algo' : LinearRegression(),
        'parameters' : {
            'linearregression__fit_intercept':[True,False], 
            'linearregression__normalize':[True,False], 
            'linearregression__copy_X':[True, False]
        }
    },
    'Ridge' : {
        'algo' : Ridge(),
        'parameters' : {
            'ridge__alpha':np.logspace(-5, 5, 10) 
        }
    },
    'Lasso' : {
        'algo' : Lasso(tol=0.5),
        'parameters' : {
            'lasso__alpha':np.logspace(-5, 5, 10) 
        }
    },
    'ElasticNet' : {
        'algo' : ElasticNet(max_iter=10000,tol=100),
        'parameters' : {
            'elasticnet__alpha':np.logspace(-5, 5, 10),  
            'elasticnet__l1_ratio':np.logspace(-5, 1, 10)
        }
    },
    'RandomForestRegressor' : {
        'algo' : RandomForestRegressor(),
        'parameters' : {
            'randomforestregressor__n_estimators': [100, 200, 300], 
            'randomforestregressor__max_depth': [None, 1, 2, 3], 
            'randomforestregressor__min_samples_split': [1, 2, 3]
        }
    },
    'XGBRegressor' : {
        'algo' : XGBRegressor(),
        'parameters' : { 
            'xgbregressor__learning_rate': [0.1, 0.5, 1],
            'xgbregressor__max_depth': [None, 4, 8],
            'xgbregressor__min_child_weight': [None, 3, 5],
            'xgbregressor__n_estimators': [100, 200, 1000]
        }
    }
}


# <a id="model-compare">Comparaison des modèles<a/> 

In [ ]:
for algo_name, algo in algos.items():
    print('Algorithme: ',algo_name)
    for column in y_columns:
        y_test = y_test_all[column]
        y_train = y_train_all[column]
        model = make_pipeline(preprocessor,algo['algo'])
        parameters = algo['parameters']
        grid = GridSearchCV(model,parameters, cv=5, n_jobs = 5)
        grid.fit(X_train, y_train)
        y_pred = grid.predict(X_test)
        print('Prédiction de ',column)
        print ("best score: ", grid.best_score_)
        print("best params: ",grid.best_params_)
        print("score de la prédiction: ",  grid.best_estimator_.score(X_test, y_test)), 
        mae = mean_absolute_error(y_test,y_pred)
        rmse = np.sqrt(mean_squared_error(y_test,y_pred))
        med_abs_err = median_absolute_error(y_test,y_pred)
        print("MAE = ", mae)        
        print("RMSE = ",rmse)
        print("median abs err = ", med_abs_err)
        print('')
        results.append([algo_name, column, grid.best_estimator_.score(X_test, y_test),mae,rmse,med_abs_err])
    print('-'*100)

# <a id="model-compare-without-ESS">Comparaison des modèles sans le EnergyStarScore<a/> 

In [ ]:
X_train = X_train.drop(['ENERGYSTARScore'], axis=1)
X_test = X_test.drop(['ENERGYSTARScore'], axis=1)
results_without_energyStarScore = []
numericColumns.remove('ENERGYSTARScore')
print(numericColumns)
preprocessor = make_column_transformer((RobustScaler(),numericColumns),(OneHotEncoder(handle_unknown = 'ignore'),objectColumns))

In [ ]:
for algo_name, algo in algos.items():
    print('algorithme: ',algo_name)
    for column in y_columns:
        y_test = y_test_all[column]
        y_train = y_train_all[column]
        model = make_pipeline(preprocessor,algo['algo'])
        parameters = algo['parameters']
        grid = GridSearchCV(model,parameters, cv=5, n_jobs = 5)
        grid.fit(X_train, y_train)
        y_pred = grid.predict(X_test)
        print('Prédiction de ',column)
        print ("best score: ", grid.best_score_)
        print("best params: ",grid.best_params_)
        print("score de la prédiction: ",  grid.best_estimator_.score(X_test, y_test)), 
        mae = mean_absolute_error(y_test,y_pred)
        rmse = np.sqrt(mean_squared_error(y_test,y_pred))
        med_abs_err = median_absolute_error(y_test,y_pred)
        print("MAE = ", mae)        
        print("RMSE = ",rmse)
        print("median abs err = ", med_abs_err)
        print('')
        results_without_energyStarScore.append([algo_name, column, grid.best_estimator_.score(X_test, y_test),mae,rmse,med_abs_err])
    print('-'*100)

# <a id="conclude">Conclusion<a/> 

In [ ]:
df_results = pd.DataFrame(results,columns=['algorithm', 'column', 'predict score', 'MAE', 'RMSE', 'median abs err'])
display(df_results.sort_values(by=['column','predict score'],ascending=False))
print("Sans le EnergyStarScore:")
df_results_without_energyStarScore = pd.DataFrame(results_without_energyStarScore,columns=['algorithm', 'column', 'predict score', 'MAE', 'RMSE', 'median abs err'])
display(df_results_without_energyStarScore.sort_values(by=['column','predict score'],ascending=False))